In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import WordPunctTokenizer
import pickle

Training set

In [2]:
address_abstracts = './Dataset/chemdner_corpus/training.abstracts.txt'

with open(address_abstracts) as f:
    
    id_nums = list()
    titles = list()
    abstracts = list()
    texts = list()
    
    for x in f.read().split('\n'):
        id_num = x.split('\t')[0]
        #print(id_num)
        title = x.split('\t')[1].rstrip()
        
        if title.endswith('?'):
            title = title[:-1] + '.'
        elif title.endswith('-'):
            title = title[:-1] + '.'
        elif title.endswith('!'):
            title = title[:-1] + '.'
        else:
            title = title
        
        abstract = x.split('\t')[2].rstrip()
        if abstract.endswith("'"):
            abstract = abstract + '.'
        else:
            abstract = abstract
        
        text = title + " " + abstract
        id_nums.append(id_num)
        titles.append(title)
        abstracts.append(abstract)
        texts.append(text)
    
    df_text_train = pd.DataFrame(list(zip(id_nums, titles, abstracts, texts)),
               columns =['ID', 'TITLE', 'ABSTRACT', 'TEXT'])
    df_text_train = df_text_train.set_index('ID')

In [4]:
address_annotations = './Dataset/chemdner_corpus/training.annotations.txt'

with open(address_annotations) as f:
    id_nums = list()
    poceci = list()
    krajevi = list()
    NERs = list()
    tags = list()
    for x in f.read().split('\n'):
        id_num, tip, pocetak_str, kraj_str, NER, tag = x.split('\t')
        pocetak = int(pocetak_str)
        kraj = int(kraj_str)
        id_nums.append(id_num)
        #tipovi.append(tip)
        if tip == 'T':
            poceci.append(pocetak)
            krajevi.append(kraj)
        else:
            poceci.append(pocetak+len(df_text_train.at[id_num, 'TITLE'])+1)
            krajevi.append(kraj+len(df_text_train.at[id_num, 'TITLE'])+1)
            
        NERs.append(NER)
        if tag == 'FAMILY':
            tags.append('Chemical Family')
        else:
            tags.append('Chemical')
    df_ann_train = pd.DataFrame(list(zip(id_nums, poceci, krajevi, NERs, tags)),
               columns =['ID', 'ONSET', 'OFFSET', 'NER', 'TAG'])

In [6]:
id_nums = list()
ts = list()
original_texts = list()
texts = list()
results = list()
original_results = list()
vectors = list()
corpuses = list()
sentences = list()
original_sentences = list()

skart = list()

for id_num in df_text_train.index:
    if id_num in ['22304302', '22516666', '22640571', '22661401', '22693035', '22780563',
                  '22809387', '22874922', '22876956', '22906572', '22955948', '22956629',
                  '22975146', '22983652', '22985735', '22985953', '22989703', '22989704',
                  '23008229', '23010362', '23010870', '23016524', '23017389', '23023136',
                  '23043441', '23046872', '23052192', '23060096', '23065696', '23071106',
                  '23092395', '23099811', '23103297', '23103426', '23116643', '23118019',
                  '23122061', '23122077', '23122140', '23131177', '23146765', '23146766',
                  '23146871', '23147377', '23147475', '23147715', '23147750', '23152186',
                  '23160419', '23160934', '23160940', '23161648', '23168915', '23174457',
                  '23178170', '23178178', '23181488', '23183413', '23192895', '23194510',
                  '23194555', '23195954', '23195960', '23198831', '23201003', '23202250',
                  '23202880', '23208666', '23212975', '23214714', '23218717', '23219590',
                  '23220514', '23223801', '23224775', '23225542', '23232941', '23233456',
                  '23234246', '23234812', '23238236', '23243659', '23245188', '23249338',
                  '23249341', '23250541', '23252481', '23254196', '23255471', '23259866',
                  '23265429', '23265526', '23266732', '23267862', '23270527', '23275110',
                  '23281937', '23284002', '23286240', '23288052', '23288103', '23289646',
                  '23289872', '23290724', '23295229', '23296100', '23296837', '23297817',
                  '23297817', '23298158', '23300000', '23300227', '23301701', '23305110',
                  '23305294', '23311718', '23311754', '23313621', '23313961', '23314277',
                  '23314757', '23315683', '23317450', '23318226', '23318905', '23320521',
                  '23321916', '23323854', '23325050', '23325631', '23327112', '23327360',
                  '23334583', '23335833', '23337202', '23338525', '23339572', '23339604',
                  '23340332', '23340646', '23343117', '23343325', '23344975', '23347684',
                  '23348152', '23350945', '23353816', '23357362', '23357629', '23361517',
                  '23361587', '23362239', '23363574', '23371763', '23373444', '23373590',
                  '23373772', '23374865', '23376119', '23376245', '23376354', '23376696',
                  '23376817', '23377100', '23379686', '23379784', '23380477', '23386250',
                  '23386597', '23387865', '23389045', '23389625', '23391162', '23391631',
                  '23391637', '23394309', '23395663', '23400262', '23400925', '23402364',
                  '23402638', '23402647', '23406277', '23408434', '23409842', '23409952',
                  '23410078', '23411082', '23411220', '23411232', '23411250', '23411268',
                  '23411342', '23412396', '23412992', '23418960', '23420400', '23421743',
                  '23425339', '23427063', '23432095', '23448384', '23452254', '23453067',
                  '23454309', '23455312', '23456038', '23457119', '23458385', '23459233',
                  '23462212', '23464789', '23464990', '23465072', '23466228', '23468426',
                  '23470152', '23474321', '23477419', '23477623', '23493374', '23495151',
                  '23496349', '23497876', '23503927', '23509853', '23510479', '23511016',
                  '23512927', '23517474', '23517546', '23518599', '23520074', '23524254',
                  '23525786', '23526231', '23527619', '23529898', '23535321', '23537133',
                  '23541637', '23543460', '23546003', '23552398', '23561157', '23562245',
                  '23570840', '23583454', '23584426', '23584427', '23585235', '23589462',
                  '23595510', '23597133', '23597793', '23598904', '23601709', '23602989',
                  '23609128', '23609438', '23611338', '23613425', '23614571', '23617226',
                  '23620162', '23625887', '23625905', '23627597', '23627902', '23632082',
                  '23638985', '23639545', '23640471', '23644192']:
        continue
    else:
        id_num = id_num
    original_text = df_text_train.at[id_num, 'TEXT']
    for t in set(tags):
        text = original_text
        pomeraj = 0
        for j in df_ann_train[df_ann_train['ID'] == id_num].index.values:
            pocetni = df_ann_train.at[j,'ONSET'] - pomeraj
            krajnji = df_ann_train.at[j,'OFFSET'] - pomeraj
            if df_ann_train.at[j,'TAG'] == t:
                #l = len(df_ann_train.at[j,'NER'].split())
                NER_len = len(WordPunctTokenizer().tokenize(df_ann_train.at[j,'NER']))
                if NER_len == 1:
                    a = 'xxxxx'
                else:
                    a = 'xxxxx'+' yyyyy'*(NER_len-1)
                        
                text = text[:pocetni] + a + text[krajnji:]
                pomeraj += krajnji - pocetni - len(a)
        
        result = WordPunctTokenizer().tokenize(text)
        original_result = WordPunctTokenizer().tokenize(original_text)
        if len(result)==len(original_result):
            pass
        else:
            skart.append(id_num)
            print(id_num)
        n = 0
        for m in original_result:
            if m == '.' or m == ').' or m == "'." or m == '".':
                k = original_result.index(m, n)
                sentence = result[n:k+1]
                original_sentence = original_result[n:k+1]
                n = k + 1
                vector = list()
                for i in sentence:
                    if i=='xxxxx':
                        i=1
                    elif i=='yyyyy':
                        i=1
                    else:
                        i=0
                    vector.append(i)
                corpus = 'CHEMDNER'
                if len(vector) >= 3:
                    id_nums.append(id_num)
                    ts.append(t)
                    titles.append(title)
                    abstracts.append(abstract)
                    original_texts.append(original_text)
                    texts.append(text)
                    results.append(result)
                    original_results.append(original_result)
                    corpuses.append(corpus)
                    sentences.append(sentence)
                    original_sentences.append(original_sentence)
                    vectors.append(vector)

df_train = pd.DataFrame(list(zip(corpuses, id_nums, ts, original_sentences, vectors)),
               columns =['cor', 'ids', 'class', 'text', 'labels'])
df_train = df_train.set_index('ids')

In [8]:
df_train

,cor,class,text,labels
ids,,,,
21826085,CHEMDNER,Chemical Family,"[DPP6, as, a, candidate, gene, for, neurolepti...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
21826085,CHEMDNER,Chemical Family,"[We, implemented, a, two, -, step, approach, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
21826085,CHEMDNER,Chemical Family,"[First, ,, we, screened, associations, by, usi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
21826085,CHEMDNER,Chemical Family,"[Next, ,, we, performed, a, replication, analy...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
21826085,CHEMDNER,Chemical Family,"[An, association, of, an, SNP, in, the, DPP6, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
23645249,CHEMDNER,Chemical,"[MSF, cultured, in, differentiation, medium, w...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
23645249,CHEMDNER,Chemical,"[Genistein, inhibition, of, adipose, different...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
23645249,CHEMDNER,Chemical,"[Reduction, of, ERβ, expression, by, siRNA, ta...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


Development set

In [9]:
address_abstracts = './Dataset/chemdner_corpus/development.abstracts.txt'

with open(address_abstracts) as f:
    
    id_nums = list()
    titles = list()
    abstracts = list()
    texts = list()
    
    for x in f.read().split('\n'):
        id_num = x.split('\t')[0]
        #print(id_num)
        title = x.split('\t')[1].rstrip()
        
        if title.endswith('?'):
            title = title[:-1] + '.'
        elif title.endswith('-'):
            title = title[:-1] + '.'
        elif title.endswith('!'):
            title = title[:-1] + '.'
        else:
            title = title
        
        abstract = x.split('\t')[2].rstrip()
        if abstract.endswith("'"):
            abstract = abstract + '.'
        else:
            abstract = abstract
        
        text = title + " " + abstract
        id_nums.append(id_num)
        titles.append(title)
        abstracts.append(abstract)
        texts.append(text)
    
    df_text_dev = pd.DataFrame(list(zip(id_nums, titles, abstracts, texts)),
               columns =['ID', 'TITLE', 'ABSTRACT', 'TEXT'])
    df_text_dev = df_text_dev.set_index('ID')

In [11]:
address_annotations = './Dataset/chemdner_corpus/development.annotations.txt'

with open(address_annotations) as f:
    id_nums = list()
    poceci = list()
    krajevi = list()
    NERs = list()
    tags = list()
    for x in f.read().split('\n'):
        id_num, tip, pocetak_str, kraj_str, NER, tag = x.split('\t')
        pocetak = int(pocetak_str)
        kraj = int(kraj_str)
        id_nums.append(id_num)
        #tipovi.append(tip)
        if tip == 'T':
            poceci.append(pocetak)
            krajevi.append(kraj)
        else:
            poceci.append(pocetak+len(df_text_dev.at[id_num, 'TITLE'])+1)
            krajevi.append(kraj+len(df_text_dev.at[id_num, 'TITLE'])+1)
            
        NERs.append(NER)
        if tag == 'FAMILY':
            tags.append('Chemical Family')
        else:
            tags.append('Chemical')
    df_ann_dev = pd.DataFrame(list(zip(id_nums, poceci, krajevi, NERs, tags)),
               columns =['ID', 'ONSET', 'OFFSET', 'NER', 'TAG'])

In [13]:
id_nums = list()
ts = list()
original_texts = list()
texts = list()
results = list()
original_results = list()
vectors = list()
corpuses = list()
sentences = list()
original_sentences = list()

skart = list()

for id_num in df_text_dev.index:
    #id_num = i
    if id_num in ['22313530', '22508482', '22564442', '22578098', '22614004', '22720714',
                  '22910795', '22977167', '22982007', '22989414', '22995439', '23019138',
                  '23022324', '23042952', '23059508', '23060411', '23072367', '23084046',
                  '23092225', '23093023', '23093494', '23104246', '23108606', '23122111',
                  '23122161', '23129806', '23136062', '23146695', '23146698', '23146755',
                  '23147639', '23150067', '23157634', '23159885', '23167767', '23172734',
                  '23174144', '23174748', '23175103', '23178257', '23178681', '23178950',
                  '23182946', '23190033', '23194532', '23197401', '23197773', '23201180',
                  '23202372', '23202442', '23203941', '23204517', '23206276', '23208822',
                  '23211363', '23212047', '23213199', '23216107', '23220641', '23229057',
                  '23229057', '23229510', '23235156', '23238642', '23240995', '23240995',
                  '23241082', '23244162', '23244594', '23244683', '23246469', '23247722',
                  '23254253', '23255254', '23255426', '23256531', '23258566', '23259487',
                  '23259819', '23260350', '23262562', '23265191', '23265463', '23265464',
                  '23265515', '23271679', '23273260', '23288867', '23290048', '23291323',
                  '23294378', '23295385', '23296027', '23297250', '23298613', '23302007',
                  '23303717', '23305513', '23306106', '23306206', '23314275', '23314308',
                  '23315522', '23317164', '23317370', '23323640', '23323975', '23327546',
                  '23327691', '23335817', '23338044', '23340670', '23343190', '23343346',
                  '23343565', '23348235', '23350627', '23350926', '23351043', '23351169',
                  '23353751', '23353756', '23353805', '23359599', '23360082', '23360333',
                  '23368735', '23368753', '23369072', '23373428', '23373704', '23382024',
                  '23384448', '23386702', '23395171', '23395804', '23396185', '23398572',
                  '23400048', '23401165', '23401298', '23401298', '23402292', '23402729',
                  '23403016', '23406949', '23409871', '23409951', '23410094', '23411077',
                  '23411205', '23411241', '23411293', '23411507', '23414214', '23414340',
                  '23418369', '23422225', '23422569', '23422727', '23425969', '23432060',
                  '23432062', '23432093', '23435367', '23435541', '23435915', '23436442',
                  '23441977', '23442788', '23443513', '23444386', '23448247', '23453838',
                  '23454112', '23459473', '23465048', '23470287', '23471850', '23473535',
                  '23474903', '23474903', '23474905', '23475307', '23481176', '23481651',
                  '23482500', '23484958', '23489627', '23504441', '23515940', '23517479',
                  '23523665', '23523950', '23525158', '23530626', '23530926', '23532002',
                  '23532057', '23537160', '23537702', '23541674', '23542495', '23542819',
                  '23548055', '23549985', '23551255', '23560438', '23561184', '23562376',
                  '23562926', '23567043', '23570261', '23571664', '23574310', '23574435',
                  '23584984', '23588252', '23591115', '23591590', '23594342', '23598368',
                  '23603044', '23608239', '23608240', '23613484', '23621524', '23621524',
                  '23621573', '23625804', '23627311', '23627699', '23629861', '23631539',
                  '23631539', '23634668', '23642033', '23643932']:
        continue
    else:
        id_num = id_num
    original_text = df_text_dev.at[id_num, 'TEXT']
    for t in set(tags):
        text = original_text
        pomeraj = 0
        for j in df_ann_dev[df_ann_dev['ID'] == id_num].index.values:
            pocetni = df_ann_dev.at[j,'ONSET'] - pomeraj
            krajnji = df_ann_dev.at[j,'OFFSET'] - pomeraj
            if df_ann_dev.at[j,'TAG'] == t:
                #l = len(df_ann_dev.at[j,'NER'].split())
                NER_len = len(WordPunctTokenizer().tokenize(df_ann_dev.at[j,'NER']))
                if NER_len == 1:
                    a = 'xxxxx'
                else:
                    a = 'xxxxx'+' yyyyy'*(NER_len-1)
                        
                text = text[:pocetni] + a + text[krajnji:]
                pomeraj += krajnji - pocetni - len(a)
        
        result = WordPunctTokenizer().tokenize(text)
        original_result = WordPunctTokenizer().tokenize(original_text)
        if len(result)==len(original_result):
            pass
        else:
            skart.append(id_num)
            print(id_num)
        n = 0
        for m in original_result:
            if m == '.' or m == ').' or m == "'." or m == '".':
                k = original_result.index(m, n)
                sentence = result[n:k+1]
                original_sentence = original_result[n:k+1]
                n = k + 1
                vector = list()
                for i in sentence:
                    if i=='xxxxx':
                        i=1
                    elif i=='yyyyy':
                        i=1
                    else:
                        i=0
                    vector.append(i)
                corpus = 'CHEMDNER'
                if len(vector) >= 3:
                    id_nums.append(id_num)
                    ts.append(t)
                    titles.append(title)
                    abstracts.append(abstract)
                    original_texts.append(original_text)
                    texts.append(text)
                    results.append(result)
                    original_results.append(original_result)
                    corpuses.append(corpus)
                    sentences.append(sentence)
                    original_sentences.append(original_sentence)
                    vectors.append(vector)

df_dev = pd.DataFrame(list(zip(corpuses, id_nums, ts, original_sentences, vectors)),
               columns =['cor', 'ids', 'class', 'text', 'labels'])
df_dev = df_dev.set_index('ids')

In [15]:
df_dev

,cor,class,text,labels
ids,,,,
21933113,CHEMDNER,Chemical Family,"[Models, to, predict, intestinal, absorption, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
21933113,CHEMDNER,Chemical Family,"[Prediction, of, human, intestinal, absorption...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
21933113,CHEMDNER,Chemical Family,"[There, are, various, techniques, currently, e...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
21933113,CHEMDNER,Chemical Family,"[Screening, protocols, to, evaluate, protein, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
21933113,CHEMDNER,Chemical Family,"[It, is, the, careful, and, critical, use, of,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
23646356,CHEMDNER,Chemical,"[Electron, transmission, spectroscopy, (, ETS,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
23646356,CHEMDNER,Chemical,"[Formation, of, long, -, lived, parent, molecu...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
23646356,CHEMDNER,Chemical,"[The, gas, -, phase, DEA, process, ,, analogou...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


Evaluation set

In [16]:
address_abstracts = './Dataset/chemdner_corpus/evaluation.abstracts.txt'

with open(address_abstracts) as f:
    
    id_nums = list()
    titles = list()
    abstracts = list()
    texts = list()
    
    for x in f.read().split('\n'):
        id_num = x.split('\t')[0]
        #print(id_num)
        title = x.split('\t')[1].rstrip()
        
        if title.endswith('?'):
            title = title[:-1] + '.'
        elif title.endswith('-'):
            title = title[:-1] + '.'
        elif title.endswith('!'):
            title = title[:-1] + '.'
        else:
            title = title
        
        abstract = x.split('\t')[2].rstrip()
        if abstract.endswith("'"):
            abstract = abstract + '.'
        else:
            abstract = abstract
        
        text = title + " " + abstract
        id_nums.append(id_num)
        titles.append(title)
        abstracts.append(abstract)
        texts.append(text)
    
    df_text_eval = pd.DataFrame(list(zip(id_nums, titles, abstracts, texts)),
               columns =['ID', 'TITLE', 'ABSTRACT', 'TEXT'])
    df_text_eval = df_text_eval.set_index('ID')

In [18]:
address_annotations = './Dataset/chemdner_corpus/evaluation.annotations.txt'

with open(address_annotations) as f:
    id_nums = list()
    poceci = list()
    krajevi = list()
    NERs = list()
    tags = list()
    for x in f.read().split('\n'):
        id_num, tip, pocetak_str, kraj_str, NER, tag = x.split('\t')
        pocetak = int(pocetak_str)
        kraj = int(kraj_str)
        id_nums.append(id_num)
        #tipovi.append(tip)
        if tip == 'T':
            poceci.append(pocetak)
            krajevi.append(kraj)
        else:
            poceci.append(pocetak+len(df_text_eval.at[id_num, 'TITLE'])+1)
            krajevi.append(kraj+len(df_text_eval.at[id_num, 'TITLE'])+1)
            
        NERs.append(NER)
        if tag == 'FAMILY':
            tags.append('Chemical Family')
        else:
            tags.append('Chemical')
    df_ann_eval = pd.DataFrame(list(zip(id_nums, poceci, krajevi, NERs, tags)),
               columns =['ID', 'ONSET', 'OFFSET', 'NER', 'TAG'])

In [20]:
id_nums = list()
ts = list()
original_texts = list()
texts = list()
results = list()
original_results = list()
vectors = list()
corpuses = list()
sentences = list()
original_sentences = list()

skart = list()

for id_num in df_text_eval.index:
    if id_num in ['23208951', '23086899', '23223191', '23298241', '23454016', '23321274',
                  '23583884', '23460404', '23594297', '23194824', '23210706', '23210706',
                  '23383865', '23601402', '23092130', '23127917', '23561193', '23391297',
                  '22294440', '22933114', '23117572', '23292193', '23212442', '23630181',
                  '23519881', '23521048', '23431112', '23608635', '23142577', '23103450',
                  '23125001', '23549423', '23219524', '23246471', '23431964', '23411291',
                  '23636270', '23141189', '23361947', '23065140', '23289972', '23411207',
                  '23160931', '23299759', '23416700', '23252447', '23178655', '23362800',
                  '23579166', '23220105', '23090712', '23244296', '23639801', '23249274',
                  '23017422', '23258482', '23211362', '23104429', '23606373', '22893354',
                  '23449501', '23301860', '23477486', '23268606', '23460473', '23187066',
                  '23150983', '23256867', '22659408', '23473268', '23411292', '23123250',
                  '22659090', '23556340', '23027385', '23537388', '22921426', '23008503',
                  '23469759', '23385876', '23385876', '22884720', '23296256', '23159914',
                  '23240892', '23143040', '23477570', '23002036', '23331245', '23089249',
                  '23541371', '23448129', '23523531', '23240866', '23265540', '23522835',
                  '22075688', '23410589', '23289770', '23254551', '23581996', '22335791',
                  '23137047', '23335435', '23351005', '23500116', '23330972', '23085190',
                  '23228182', '23423713', '23564313', '23564313', '23411329', '23298334',
                  '23478437', '23583933', '23518229', '23266269', '23608535', '23387414',
                  '23387414', '23292321', '23295712', '23438015', '23063001', '23534826',
                  '23265513', '23465061', '22885792', '23138334', '23537583', '23534618',
                  '23330726', '23573951', '23165625', '23223414', '23266270', '23220410',
                  '22717325', '23552828', '23437929', '23250742', '23297412', '23474029',
                  '23381223', '23538867', '22963374', '23087260', '23480174', '23514045',
                  '23202888', '23259935', '23092949', '23340651', '23023768', '23465613',
                  '23316943', '23524312', '23402300', '23529672', '23252343', '23331095',
                  '23434418', '23249238', '23611819', '22767283', '23617758', '23617758',
                  '23411247', '23541843', '23327779', '23574336', '23583931', '23427814',
                  '23341452', '23485017', '23301836', '23400365', '23261716', '23588092',
                  '23228444', '23497903', '23265535', '23474593', '23360537', '23211008',
                  '23344693', '23295222', '23562533', '23584844', '23524663', '22943130',
                  '23210621', '23335392', '23454217', '23510527', '23410068', '23418898']:
        continue
    else:
        id_num = id_num
    original_text = df_text_eval.at[id_num, 'TEXT']
    for t in set(tags):
        text = original_text
        pomeraj = 0
        for j in df_ann_eval[df_ann_eval['ID'] == id_num].index.values:
            pocetni = df_ann_eval.at[j,'ONSET'] - pomeraj
            krajnji = df_ann_eval.at[j,'OFFSET'] - pomeraj
            if df_ann_eval.at[j,'TAG'] == t:
                #l = len(df_ann_eval.at[j,'NER'].split())
                NER_len = len(WordPunctTokenizer().tokenize(df_ann_eval.at[j,'NER']))
                if NER_len == 1:
                    a = 'xxxxx'
                else:
                    a = 'xxxxx'+' yyyyy'*(NER_len-1)
                        
                text = text[:pocetni] + a + text[krajnji:]
                pomeraj += krajnji - pocetni - len(a)
        
        result = WordPunctTokenizer().tokenize(text)
        original_result = WordPunctTokenizer().tokenize(original_text)
        if len(result)==len(original_result):
            pass
        else:
            skart.append(id_num)
            print(id_num)
        n = 0
        for m in original_result:
            if m == '.' or m == ').' or m == "'." or m == '".':
                k = original_result.index(m, n)
                sentence = result[n:k+1]
                original_sentence = original_result[n:k+1]
                n = k + 1
                vector = list()
                for i in sentence:
                    if i=='xxxxx':
                        i=1
                    elif i=='yyyyy':
                        i=1
                    else:
                        i=0
                    vector.append(i)
                corpus = 'CHEMDNER'
                if len(vector) >= 3:
                    id_nums.append(id_num)
                    ts.append(t)
                    titles.append(title)
                    abstracts.append(abstract)
                    original_texts.append(original_text)
                    texts.append(text)
                    results.append(result)
                    original_results.append(original_result)
                    corpuses.append(corpus)
                    sentences.append(sentence)
                    original_sentences.append(original_sentence)
                    vectors.append(vector)

df_eval = pd.DataFrame(list(zip(corpuses, id_nums, ts, original_sentences, vectors)),
               columns =['cor', 'ids', 'class', 'text', 'labels'])
df_eval = df_eval.set_index('ids')

In [22]:
df_eval

,cor,class,text,labels
ids,,,,
23323548,CHEMDNER,Chemical Family,"[Cloning, and, functional, characterization, o...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
23323548,CHEMDNER,Chemical Family,"[Abstract, 1, .]","[0, 0, 0]"
23323548,CHEMDNER,Chemical Family,"[Organic, anion, transporting, polypeptides, (...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
23323548,CHEMDNER,Chemical Family,"[Human, OATP1A2, has, been, demonstrated, to, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
23323548,CHEMDNER,Chemical Family,"[Study, on, OATP1A2, orthologues, of, other, s...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...
23343596,CHEMDNER,Chemical,"[Subjective, responses, to, the, drug, were, m...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
23343596,CHEMDNER,Chemical,"[Drug, responses, were, reduced, via, principa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
23343596,CHEMDNER,Chemical,"[High, trait, physical, fearlessness, was, sig...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."


In [23]:
df = pd.concat([df_train, df_dev, df_eval])

In [24]:
df['labels'] = df['labels'].apply(lambda x: np.array(x))
df['labels'] = df['labels'].apply(lambda x: x.astype(np.int64))
df['class'] = df['class'].apply(lambda x: x.split())

In [26]:
df = df.reset_index()
df

,ids,cor,class,text,labels
0,21826085,CHEMDNER,"[Chemical, Family]","[DPP6, as, a, candidate, gene, for, neurolepti...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,21826085,CHEMDNER,"[Chemical, Family]","[We, implemented, a, two, -, step, approach, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,21826085,CHEMDNER,"[Chemical, Family]","[First, ,, we, screened, associations, by, usi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,21826085,CHEMDNER,"[Chemical, Family]","[Next, ,, we, performed, a, replication, analy...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,21826085,CHEMDNER,"[Chemical, Family]","[An, association, of, an, SNP, in, the, DPP6, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
190437,23343596,CHEMDNER,[Chemical],"[Subjective, responses, to, the, drug, were, m...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
190438,23343596,CHEMDNER,[Chemical],"[Drug, responses, were, reduced, via, principa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
190439,23343596,CHEMDNER,[Chemical],"[High, trait, physical, fearlessness, was, sig...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
190440,23343596,CHEMDNER,[Chemical],"[In, addition, ,, high, trait, impulsivity, wa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [27]:
# using protocol 4 for dumping a .pkl file

filename = './CHEMDNER_protokol4.pkl'
outfile = open(filename,'wb')
pickle.dump(df,outfile, protocol=4)